In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD

# Load the datasets
stock_data = pd.read_csv("Stock_Data.csv")
trader_data = pd.read_csv("Trader_Data.csv", index_col=0)

# Step 1: Preprocess the Data
# Normalize numerical features
numerical_features = ["Market_Cap", "P/E_Ratio", "Dividend_Yield", "Volatility", "Momentum", "Sentiment_Score"]
scaler = MinMaxScaler()
stock_data[numerical_features] = scaler.fit_transform(stock_data[numerical_features])

# One-hot encode the Sector column
stock_data = pd.get_dummies(stock_data, columns=["Sector"])

# Step 2: Content-Based Filtering
def recommend_similar_assets(asset_id, top_n=5):
    # Extract feature vectors
    features = stock_data.set_index("Asset").drop(columns="Asset").values
    # Compute cosine similarity
    similarity_matrix = cosine_similarity(features)
    asset_index = stock_data.index[stock_data["Asset"] == asset_id].tolist()[0]
    similar_indices = similarity_matrix[asset_index].argsort()[::-1][1:top_n + 1]
    return stock_data.iloc[similar_indices]["Asset"]

# Example: Recommend similar assets to Asset_1
print("Content-Based Recommendations for Asset_1:")
print(recommend_similar_assets("Asset_1"))

# Step 3: Collaborative Filtering
# Convert trader data to user-item matrix
user_item_matrix = trader_data.T  # Transpose for user-item structure

# Apply Singular Value Decomposition (SVD)
svd = TruncatedSVD(n_components=10, random_state=42)
latent_matrix = svd.fit_transform(user_item_matrix)
item_latent_matrix = svd.components_.T

# Predict missing values
predicted_interactions = np.dot(latent_matrix, item_latent_matrix.T)
predicted_df = pd.DataFrame(predicted_interactions, index=user_item_matrix.index, columns=user_item_matrix.columns)

def recommend_to_trader(trader_id, top_n=5):
    trader_ratings = predicted_df.loc[trader_id]
    recommended_assets = trader_ratings.sort_values(ascending=False).index[:top_n]
    return recommended_assets

# Example: Recommend assets to Trader_1
print("Collaborative Recommendations for Trader_1:")
print(recommend_to_trader("Trader_1"))

# Step 4: Hybrid Filtering
def hybrid_recommendation(trader_id, asset_id, weight_content=0.5, weight_collaborative=0.5, top_n=5):
    # Content-based similarity
    content_recommendations = recommend_similar_assets(asset_id, top_n=len(stock_data))
    content_scores = pd.Series(
        [1 / (i + 1) for i in range(len(content_recommendations))],
        index=content_recommendations
    )
    
    # Collaborative scores
    collaborative_scores = predicted_df.loc[trader_id]
    
    # Combine scores
    combined_scores = content_scores * weight_content + collaborative_scores * weight_collaborative
    return combined_scores.nlargest(top_n).index

# Example: Hybrid recommendation for Trader_1 and Asset_1
print("Hybrid Recommendations for Trader_1 with Asset_1 as input:")
print(hybrid_recommendation("Trader_1", "Asset_1"))

# Step 5: Evaluation
# Split data into train and test sets
train_data = trader_data.sample(frac=0.8, random_state=42)
test_data = trader_data.drop(train_data.index)

# Calculate precision, recall, or backtest (to be customized for your needs)

# Step 6: Deploy the Model
# Build a dashboard or API for deployment
